In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import joblib

In [2]:
pd.pandas.set_option('display.max_columns', None) #habilitamos despliegue maximo de columnas

In [3]:
data = pd.read_csv('customer_data_edited.csv')
print(data.shape)

(12892, 22)


In [4]:
data.head()

,recordID,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,customer_id
0,1,HI,101,510,no,no,0,70.9,123,12.05,211.9,73,18.01,236.0,73,10.62,10.6,3,2.86,3,no,23383607.0
1,2,MT,137,510,no,no,0,223.6,86,38.01,244.8,139,20.81,94.2,81,4.24,9.5,7,2.57,0,no,22550362.0
2,3,OH,103,408,no,yes,29,294.7,95,50.10,237.3,105,20.17,300.3,127,13.51,13.7,6,3.70,1,no,59063354.0
3,4,NM,99,415,no,no,0,216.8,123,36.86,126.4,88,10.74,220.6,82,9.93,15.7,2,4.24,1,no,25464504.0
4,5,SC,108,415,no,no,0,197.4,78,33.56,124.0,101,10.54,204.5,107,9.20,7.7,4,2.08,2,no,691824.0


In [5]:
#Separamos data para entrenamiento y prueba,
X_train, X_test, y_train, y_test = train_test_split(
                        data.drop(['recordID', 'customer_id'], axis=1),
                        data['churn'],
                        test_size=0.15,
                        random_state=2021)

In [6]:
X_train.shape, X_test.shape

((10958, 20), (1934, 20))

## 2. Balanceo de Datos

#### 2.1 X_train

In [7]:
X_train['churn'].value_counts()

no     9417
yes    1541
Name: churn, dtype: int64

In [8]:
dataNegativa=X_train[X_train['churn']=='no']
dataNegativa.shape

(9417, 20)

In [9]:
dataPositiva=X_train[X_train['churn']=='yes']
dataPositiva.shape

(1541, 20)

In [10]:
cantidadDataNegativa=2*dataPositiva.shape[0]

In [11]:
dataNegativa = dataNegativa.sample(n=cantidadDataNegativa,random_state=2021)
dataNegativa.shape

(3082, 20)

In [12]:
X_train=pd.concat([dataPositiva,dataNegativa])
X_train.shape

(4623, 20)

In [13]:
X_train = X_train.drop(['churn'], axis=1)

In [14]:
X_train.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
1523,TX,14,415,no,no,0,271.9,104,46.22,171.4,104,14.57,239.2,111,10.76,13.8,4,3.73,2
694,WY,67,510,no,no,0,125.0,96,21.25,294.5,114,25.03,205.7,75,9.26,10.2,4,2.75,1
3766,MA,71,510,no,no,0,290.4,108,49.37,253.9,92,21.58,263.3,126,11.85,10.1,5,2.73,3
4818,ME,56,408,no,no,0,221.9,112,37.72,278.2,122,23.65,288.1,85,12.96,7.1,5,1.92,0
5975,DE,129,510,no,no,0,334.3,118,56.83,192.1,104,16.33,191.0,83,8.59,10.4,6,2.81,0


#### 2.2 X_test

In [15]:
X_test['churn'].value_counts()

no     1652
yes     282
Name: churn, dtype: int64

In [16]:
dataNegativa=X_test[X_test['churn']=='no']
dataNegativa.shape

(1652, 20)

In [17]:
dataPositiva=X_test[X_test['churn']=='yes']
dataPositiva.shape

(282, 20)

In [18]:
cantidadDataNegativa=2*dataPositiva.shape[0]

In [19]:
dataNegativa = dataNegativa.sample(n=cantidadDataNegativa,random_state=2021)
dataNegativa.shape

(564, 20)

In [20]:
X_test=pd.concat([dataPositiva,dataNegativa])
X_test.shape

(846, 20)

In [21]:
X_test = X_test.drop(['churn'], axis=1)
X_test.shape

(846, 19)

## 3. Missing Values
* La data no cuenta con valores faltantes.

## 4. Transformación de Variables Numéricas

In [22]:
#Aplicamos transoformación de Yeo-Jonhson
X_train['total_intl_calls'], param = stats.yeojohnson(X_train['total_intl_calls'])

In [23]:
X_test['total_intl_calls'] = stats.yeojohnson(X_test['total_intl_calls'], lmbda=param)

#### Binarización de Variables con Sesgo fuerte

In [24]:
sesgadas = ['number_vmail_messages']

In [25]:
for var in sesgadas:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

## 5. Codificación de Variables Categóricas

In [26]:
cat_vars = [var for var in X_train.columns if data[var].dtype == 'O']
X_train['area_code'] = X_train['area_code'].astype('O')

#Agregamos el caso especial de MSSubClass ya que por definición es categórica
cat_vars = cat_vars + ['area_code']
cat_vars

['state', 'international_plan', 'voice_mail_plan', 'area_code']

### 5.1 One Hot Encoding

In [27]:
binary_vars = ['international_plan', 'voice_mail_plan']

 ### 5.1.1. X_train

In [28]:
X_train = pd.get_dummies(X_train, columns=binary_vars, drop_first=True)
X_train

,state,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,international_plan_yes,voice_mail_plan_yes
1523,TX,14,415,0,271.9,104,46.22,171.4,104,14.57,239.2,111,10.76,13.8,1.662758,3.73,2,0,0
694,WY,67,510,0,125.0,96,21.25,294.5,114,25.03,205.7,75,9.26,10.2,1.662758,2.75,1,0,0
3766,MA,71,510,0,290.4,108,49.37,253.9,92,21.58,263.3,126,11.85,10.1,1.858008,2.73,3,0,0
4818,ME,56,408,0,221.9,112,37.72,278.2,122,23.65,288.1,85,12.96,7.1,1.858008,1.92,0,0,0
5975,DE,129,510,0,334.3,118,56.83,192.1,104,16.33,191.0,83,8.59,10.4,2.024212,2.81,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,CA,112,415,0,208.7,150,35.48,212.8,104,18.09,178.1,98,8.01,8.5,1.662758,2.30,0,0,0
7709,ME,193,415,1,71.2,58,12.10,124.7,105,10.60,155.5,108,7.00,11.7,1.425734,3.16,0,0,1
4812,LA,62,415,0,186.8,94,31.76,207.6,92,17.65,195.0,98,8.78,8.8,1.662758,2.38,3,0,0
7558,AR,94,408,0,136.2,114,23.15,165.1,118,14.03,137.9,71,6.21,9.6,1.858008,2.59,0,0,0


### 5.1.2 X_test

In [29]:
X_test = pd.get_dummies(X_test, columns=binary_vars, drop_first=True)
X_test

,state,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,international_plan_yes,voice_mail_plan_yes
8280,AR,76,408,0,107.3,140,18.24,238.2,133,20.25,271.8,116,12.23,10.0,1.425734,2.70,4,0,0
8477,VT,117,408,0,167.1,86,28.41,177.5,87,15.09,249.4,132,11.22,14.1,2.169021,3.81,2,1,0
4568,MT,85,408,1,89.8,88,15.27,233.2,75,19.82,165.7,116,7.46,9.3,2.169021,2.51,4,0,1
1986,SD,128,510,1,223.5,81,38.00,188.8,74,16.05,154.9,101,6.97,9.4,1.123285,2.54,2,1,1
5181,AK,126,415,0,58.2,94,9.89,138.7,118,11.79,136.8,91,6.16,11.9,0.702915,3.21,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8909,FL,92,415,0,201.9,74,34.32,226.8,119,19.28,217.5,80,9.79,13.7,2.024212,3.70,3,0,0
10704,IL,100,415,0,191.9,95,32.62,200.9,101,17.08,271.9,74,12.24,18.2,1.425734,4.91,1,0,0
1991,VA,129,408,0,207.0,91,35.19,154.9,121,13.17,245.1,112,11.03,13.4,1.858008,3.62,3,0,0
7671,SC,152,408,0,140.5,92,23.89,186.8,96,15.88,227.0,89,10.22,9.5,1.858008,2.57,2,0,0


### 5.2 Freq Encoding

In [30]:
def freq_map(train, test, var):
    
    train_freq_map = (train[var].value_counts().sort_values(ascending = False)).to_dict()
    test_freq_map = (test[var].value_counts().sort_values(ascending = False)).to_dict()
    
    train[var] = data[var].map(train_freq_map)
    test[var] = data[var].map(test_freq_map)

In [31]:
big_cat_vars = [var for var in cat_vars if(var not in binary_vars)]
big_cat_vars

['state', 'area_code']

In [32]:
for var in big_cat_vars:
    freq_map(X_train, X_test, var)

In [33]:
X_train

,state,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,international_plan_yes,voice_mail_plan_yes
1523,110,14,2290,0,271.9,104,46.22,171.4,104,14.57,239.2,111,10.76,13.8,1.662758,3.73,2,0,0
694,92,67,1186,0,125.0,96,21.25,294.5,114,25.03,205.7,75,9.26,10.2,1.662758,2.75,1,0,0
3766,88,71,1186,0,290.4,108,49.37,253.9,92,21.58,263.3,126,11.85,10.1,1.858008,2.73,3,0,0
4818,90,56,1147,0,221.9,112,37.72,278.2,122,23.65,288.1,85,12.96,7.1,1.858008,1.92,0,0,0
5975,80,129,1186,0,334.3,118,56.83,192.1,104,16.33,191.0,83,8.59,10.4,2.024212,2.81,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,61,112,2290,0,208.7,150,35.48,212.8,104,18.09,178.1,98,8.01,8.5,1.662758,2.30,0,0,0
7709,90,193,2290,1,71.2,58,12.10,124.7,105,10.60,155.5,108,7.00,11.7,1.425734,3.16,0,0,1
4812,71,62,2290,0,186.8,94,31.76,207.6,92,17.65,195.0,98,8.78,8.8,1.662758,2.38,3,0,0
7558,81,94,1147,0,136.2,114,23.15,165.1,118,14.03,137.9,71,6.21,9.6,1.858008,2.59,0,0,0


## 6. Scaling

In [35]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)


X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

In [39]:
X_train

,state,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,international_plan_yes,voice_mail_plan_yes
0,0.733333,0.053719,1.000000,0.0,0.773542,0.630303,0.773427,0.471268,0.611765,0.471368,0.554857,0.607362,0.554638,0.690,0.513340,0.690741,0.222222,0.0,0.0
1,0.493333,0.272727,0.034121,0.0,0.355619,0.581818,0.355589,0.809733,0.670588,0.809770,0.459143,0.386503,0.459339,0.510,0.513340,0.509259,0.111111,0.0,0.0
2,0.440000,0.289256,0.034121,0.0,0.826174,0.654545,0.826138,0.698103,0.541176,0.698156,0.623714,0.699387,0.623888,0.505,0.573620,0.505556,0.333333,0.0,0.0
3,0.466667,0.227273,0.000000,0.0,0.631294,0.678788,0.631191,0.764916,0.717647,0.765125,0.694571,0.447853,0.694409,0.355,0.573620,0.355556,0.000000,0.0,0.0
4,0.333333,0.528926,0.034121,0.0,0.951067,0.715152,0.950971,0.528183,0.611765,0.528308,0.417143,0.435583,0.416773,0.520,0.624931,0.520370,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4618,0.080000,0.458678,1.000000,0.0,0.593741,0.909091,0.593708,0.585098,0.611765,0.585247,0.380286,0.527607,0.379924,0.425,0.513340,0.425926,0.000000,0.0,0.0
4619,0.466667,0.793388,1.000000,1.0,0.202560,0.351515,0.202477,0.342865,0.617647,0.342931,0.315714,0.588957,0.315756,0.585,0.440165,0.585185,0.000000,0.0,1.0
4620,0.213333,0.252066,1.000000,0.0,0.531437,0.569697,0.531459,0.570800,0.541176,0.571013,0.428571,0.527607,0.428844,0.440,0.513340,0.440741,0.333333,0.0,0.0
4621,0.346667,0.384298,0.000000,0.0,0.387482,0.690909,0.387383,0.453946,0.694118,0.453898,0.265429,0.361963,0.265565,0.480,0.573620,0.479630,0.000000,0.0,0.0


In [36]:
#Guardamos dataset con data preparada para entrenamiento.

X_train.to_csv('preprocess_data/prep_Xtrain.csv', index=False)
X_test.to_csv('preprocess_data/prep_Xtest.csv', index=False)

y_train.to_csv('preprocess_data/prep_ytrain.csv', index=False)
y_test.to_csv('preprocess_data/prep_ytest.csv', index=False)

In [37]:
joblib.dump(scaler, 'preprocess_data/minmax_scaler.joblib')

['preprocess_data/minmax_scaler.joblib']

In [38]:
np.sum(X_train[X_train == 'Unf'].sum(axis=0))

0.0